In [1]:
import subprocess
import os
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from collections import defaultdict
import csv

In [2]:
# Define file paths

casefaa = "/home/jovyan/work/10.data_CODA_ahslyy/03.Result.SAKit2_corrected/0012934312_CA/results/05.Isoform_Novel/COAD_0012934312_CA.isoform_classification.filtered_lite.faa"
outpath=os.path.dirname(casefaa)
prefix="COAD_0012934312_CA.isoform_classification.filtered_lite.specific5"


### control of mouse
'''
ctrl_C1498_faa = "/home/jovyan/work/02.ResDev/19.USTC_C1498/03.Result_large_scale_variants/C1498_Tail/results/05.Isoform_Novel/C1498_Tail.isoform_classification.filtered_lite.faa"
ctrl_gl261_faa = "/home/jovyan/work/08.data_GBM/04.Result.large-scale.var/C57P/results/05.Isoform_Novel/C57P.isoform_classification.filtered_lite.faa"
ctrl_B16F1O_faa = "/home/jovyan/work/02.ResDev/18.USTC_B16F10/Result_B16F10_ctrl/results/05.Isoform_Novel/B16F10_ctrl.isoform_classification.filtered_lite.faa"
'''
### control of human
COAD_0009689008 = "/home/jovyan/work/10.data_CODA_ahslyy/0009689008_N/results/05.Isoform_Novel/COAD_0009689008_N.isoform_classification.filtered_lite.faa"
COAD_0012934312 = "/home/jovyan/work/10.data_CODA_ahslyy/0012934312_N/results/05.Isoform_Novel/COAD_0012934312_N.isoform_classification.filtered_lite.faa"
COAD_0014215947 = "/home/jovyan/work/10.data_CODA_ahslyy/0014215947_N/results/05.Isoform_Novel/COAD_0014215947_N.isoform_classification.filtered_lite.faa"
COAD_0014274065 = "/home/jovyan/work/10.data_CODA_ahslyy/0014274065_N/results/05.Isoform_Novel/COAD_0014274065_N.isoform_classification.filtered_lite.faa"


sp_human_faa = "/home/jovyan/work/00.database/18.uniprot/sp_human_canon_isoform.fasta"
tr_human_faa = "/home/jovyan/work/00.database/18.uniprot/tr_human_canon_isoform.fasta"
sp_mouse_faa = "/home/jovyan/work/00.database/18.uniprot/sp_mouse_canon_isoform.fasta"
tr_mouse_faa = "/home/jovyan/work/00.database/18.uniprot/tr_mouse_canon_isoform.fasta"

ideb_select_faa = os.path.join(outpath,"".join([prefix,".iedb.faa"]))
case_sepecific_faa1 = casefaa.split(".faa")[0]+".specific1.faa"
case_sepecific_faa2 = casefaa.split(".faa")[0]+".specific2.faa"
case_sepecific_faa3 = casefaa.split(".faa")[0]+".specific3.faa"
case_sepecific_faa4 = casefaa.split(".faa")[0]+".specific4.faa"
case_sepecific_faa5 = casefaa.split(".faa")[0]+".specific5.faa"
case_sepecific_faa6 = casefaa.split(".faa")[0]+".specific6.faa"

'''
ctrldedupfaa = "/home/jovyan/work/09.data_STAD_sdfyy/04.Result.LSTVs/CCSW-01-P-2/results/05.Isoform_Novel/STAD_P2.isoform_classification.filtered_lite.dedup.faa"
casededupfaa = "/home/jovyan/work/09.data_STAD_sdfyy/04.Result.LSTVs/CCSW-01-T-1/results/05.Isoform_Novel/STAD-T1.isoform_classification.filtered_lite.dedup.faa"
case8aafaa = "/home/jovyan/work/09.data_STAD_sdfyy/04.Result.LSTVs/CCSW-01-T-1/results/05.Isoform_Novel/STAD-T1.isoform_classification.filtered_lite.8.faa"
parse_blast_sh="/home/jovyan/work/09.data_STAD_sdfyy/04.Result.LSTVs/CCSW-01-T-1/results/05.Isoform_Novel/parse_blast.sh"
'''

'\nctrldedupfaa = "/home/jovyan/work/09.data_STAD_sdfyy/04.Result.LSTVs/CCSW-01-P-2/results/05.Isoform_Novel/STAD_P2.isoform_classification.filtered_lite.dedup.faa"\ncasededupfaa = "/home/jovyan/work/09.data_STAD_sdfyy/04.Result.LSTVs/CCSW-01-T-1/results/05.Isoform_Novel/STAD-T1.isoform_classification.filtered_lite.dedup.faa"\ncase8aafaa = "/home/jovyan/work/09.data_STAD_sdfyy/04.Result.LSTVs/CCSW-01-T-1/results/05.Isoform_Novel/STAD-T1.isoform_classification.filtered_lite.8.faa"\nparse_blast_sh="/home/jovyan/work/09.data_STAD_sdfyy/04.Result.LSTVs/CCSW-01-T-1/results/05.Isoform_Novel/parse_blast.sh"\n'

## 01.find cancer-specific protein sequence

In [3]:
def find_unique_sequences(ctrl_faa, case_faa, out_faa, window_size=13):
    # 读取ctrl_faa序列
    print("正在读取对照FASTA文件...")
    ctrl_sequences = set()
    for record in SeqIO.parse(ctrl_faa, "fasta"):
        seq = str(record.seq)
        for i in range(len(seq) - window_size + 1):
            ctrl_sequences.add(seq[i:i + window_size])

    # 从case_faa中找出不存在于ctrl_faa的序列
    print("正在从实验组FASTA文件中提取唯一的多肽序列...")
    unique_peptides = defaultdict(list)
    for record in SeqIO.parse(case_faa, "fasta"):
        seq = str(record.seq)
        read_id = record.id
        for i in range(len(seq) - window_size + 1):
            peptide = seq[i:i + window_size]
            if peptide not in ctrl_sequences:
                unique_peptides[read_id].append((peptide, i))

    # 合并同一read_id下的相邻多肽
    print("正在合并同一read_id下的相邻多肽序列...")
    merged_peptides = []
    for read_id, peptides in unique_peptides.items():
        merged = []
        peptides.sort(key=lambda x: x[1])  # 按坐标排序
        for peptide, start in peptides:
            if not merged or start > merged[-1][2]:
                merged.append((peptide, start, start + len(peptide)))
            else:
                overlap_start = merged[-1][2] - start
                merged_seq = merged[-1][0][:-overlap_start] + peptide
                merged[-1] = (merged_seq, merged[-1][1], start + len(peptide))

        merged_peptides.extend([(f">{read_id}_{start}_{end}", seq) for seq, start, end in merged])

    # 写入out_faa
    print(f"正在将合并后的多肽序列写入文件 {out_faa}...")
    with open(out_faa, "w") as out_file:
        for header, sequence in merged_peptides:
            out_file.write(f"{header}\n{sequence}\n")
    print(f"写入完成！")

In [4]:
# 示例用法
find_unique_sequences(COAD_0009689008, casefaa, case_sepecific_faa1)
find_unique_sequences(COAD_0012934312, case_sepecific_faa1, case_sepecific_faa2)
find_unique_sequences(COAD_0014215947, case_sepecific_faa2, case_sepecific_faa3)
find_unique_sequences(COAD_0014274065, case_sepecific_faa3, case_sepecific_faa4)
find_unique_sequences(sp_human_faa, case_sepecific_faa4, case_sepecific_faa5)
find_unique_sequences(tr_human_faa, case_sepecific_faa5, case_sepecific_faa6)

正在读取对照FASTA文件...
正在从实验组FASTA文件中提取唯一的多肽序列...
正在合并同一read_id下的相邻多肽序列...
正在将合并后的多肽序列写入文件 /home/jovyan/work/10.data_CODA_ahslyy/03.Result.SAKit2_corrected/0012934312_CA/results/05.Isoform_Novel/COAD_0012934312_CA.isoform_classification.filtered_lite.specific1.faa...
写入完成！
正在读取对照FASTA文件...
正在从实验组FASTA文件中提取唯一的多肽序列...
正在合并同一read_id下的相邻多肽序列...
正在将合并后的多肽序列写入文件 /home/jovyan/work/10.data_CODA_ahslyy/03.Result.SAKit2_corrected/0012934312_CA/results/05.Isoform_Novel/COAD_0012934312_CA.isoform_classification.filtered_lite.specific2.faa...
写入完成！
正在读取对照FASTA文件...
正在从实验组FASTA文件中提取唯一的多肽序列...
正在合并同一read_id下的相邻多肽序列...
正在将合并后的多肽序列写入文件 /home/jovyan/work/10.data_CODA_ahslyy/03.Result.SAKit2_corrected/0012934312_CA/results/05.Isoform_Novel/COAD_0012934312_CA.isoform_classification.filtered_lite.specific3.faa...
写入完成！
正在读取对照FASTA文件...
正在从实验组FASTA文件中提取唯一的多肽序列...
正在合并同一read_id下的相邻多肽序列...
正在将合并后的多肽序列写入文件 /home/jovyan/work/10.data_CODA_ahslyy/03.Result.SAKit2_corrected/0012934312_CA/results/05.Isoform_Novel/COAD_00

## 02. Select from IEDB

In [26]:
import csv
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

def select_epitope_from_IDEB(iedb_epitope, aa_file, aa_result):
    # 读取 file1.csv,获取所有表位序列
    epitopes = []
    with open(iedb_epitope, 'r') as file:
        reader = csv.reader(file)
        next(reader)  # 跳过标题行
        for row in reader:
            epitope = row[2]  # 假设第三列是表位序列
            if len(epitope) >= 7:
                epitopes.append(epitope)

    # 读取 file2.fasta,查找包含表位序列的蛋白质
    result = []
    for record in SeqIO.parse(aa_file, 'fasta'):
        sequence = str(record.seq)
        contained_epitopes = []
        for epitope in epitopes:
            if epitope in sequence:
                contained_epitopes.append(epitope)

        new_seq = Seq(sequence)  # 创建新的 Seq 对象
        if contained_epitopes:
            new_description = f"IEDB_epitope: {', '.join(contained_epitopes)}"
        else:
            new_description = "IEDB_epitope: null"

        new_record = SeqRecord(new_seq, id=record.id, description=new_description)
        result.append(new_record)

    # 将结果写入新的 fasta 文件
    with open(aa_result, 'w') as output_file:
        SeqIO.write(result, output_file, 'fasta-2line')

# 示例用法
iedb_epitope = "/home/jovyan/work/02.ResDev/22.epitope_from_idedb/CEDAR.IEDB/epitope_human_v3.csv"
aa_file = "/home/jovyan/work/10.data_CODA_ahslyy/0014274065_CA/results/09.EpitopePrediction/01.protein_sequence/COAD_0014274065.manual.AminoAcidseq.faa"
aa_result = ".".join([aa_file[:-4],"iedb","faa"])
select_epitope_from_IDEB(iedb_epitope, aa_file, aa_result)

## 03.溯源回转录本异构体

In [6]:
def reverse_sequence(sequence):
    return sequence[::-1]

def check_trans_state(exon_start,exon_end,remain_protein_seq_len):
    #print("exon_start,exon_end,remain_protein_seq_len",exon_start,exon_end,remain_protein_seq_len)
    exon_amino_length = (exon_end - exon_start)//3
    if exon_amino_length < remain_protein_seq_len:
        current_acid_index = exon_end
        remain_protein_seq_len -= exon_amino_length
        trans_started = True
        trans_overed = False
        
    elif  exon_amino_length == remain_protein_seq_len:
        current_acid_index = exon_end
        remain_protein_seq_len = 0 
        trans_started = True
        trans_overed = True 
        
    elif exon_amino_length > remain_protein_seq_len:
        current_acid_index = exon_start + remain_protein_seq_len*3
        remain_protein_seq_len = 0
        trans_started = True
        trans_overed = True 
    #print(exon_start, exon_end, remain_protein_seq_len,current_acid_index)
    return ([remain_protein_seq_len,current_acid_index,trans_started,trans_overed])

def find_amino_acid_position(exon_coordinates, translation_end, remain_protein_seq_len,strand):
    # 计算翻译的起始密码子在基因组上的起始位置
    translation_end_position = translation_end
    #print("remain_protein_seq_len",remain_protein_seq_len)
    #print("translation_end",translation_end)
    
    # 遍历exons
    trans_started = False
    trans_overed = False
    for exon_start, exon_end in exon_coordinates:
        #print(translation_end_position,exon_start, exon_end)
        exon_length = exon_end - exon_start + 1
        # 如果翻译的起始密码子在当前exon内
        
        if (trans_started == False) and (trans_overed == False):
            if translation_end_position in range(exon_start, exon_end):
                remain_protein_seq_len,current_acid_index,trans_started,trans_overed = \
                    check_trans_state(translation_end_position, exon_end, remain_protein_seq_len)
                #print("current_acid_index",current_acid_index,"trans_started",trans_started,"trans_overed",trans_overed,"remain_protein_seq_len",remain_protein_seq_len)
            else:
                print(exon_start, exon_end, translation_end_position)
                remain_protein_seq_len,current_acid_index,trans_started,trans_overed = \
                    check_trans_state(translation_end_position, exon_end, remain_protein_seq_len)
        elif (trans_started == True) and (trans_overed == False):
            remain_protein_seq_len,current_acid_index,trans_started,trans_overed = \
                    check_trans_state(exon_start, exon_end,remain_protein_seq_len)
            
        elif  (trans_started == True) and (trans_overed == True):
            break
    #print(current_acid_index)
    return current_acid_index

    
def find_isoform_info(query_seq, info,exon_coordinates):
    orf_seq = info["ORF_seq"]
    strand = info["strand"]
    cds_genomic_start = int(info["CDS_genomic_start"])
    cds_genomic_end = int(info["CDS_genomic_end"])

    #print(orf_seq,query_seq,strand,cds_genomic_start,cds_genomic_end)
    reverse_query_seq = reverse_sequence(query_seq)
    reverse_orf_seq = reverse_sequence(orf_seq)
    pep_end_pos = reverse_orf_seq.find(reverse_query_seq)
    pep_start_pos =  pep_end_pos+len(reverse_query_seq)
    '''
    # 由于原始文件中cds_genomic_start是以起始密码子为起始为点的，
    # 但是由于5'的降解，给出的蛋白质序列是认为更加完整的蛋白质序列，例如GBM_GL261细胞系检出的PB.1624.1
        KFCSYIFTNGKTKTLRGGVIVTGNRLGTLVQTYVNAINSGTVPCLENAVTTLAQRENSIAVQKAADHY
        SEQMAQRVRLPTDTLQELLTVHAACEKEAIAVFMEHSFKDDEQEFQKKLVVTIEERKEEFIRQNEAAS
        IRHCQAELERLSESLRKSISCGAFSVPGGHSLYLEARKKIELGYQQVPRKGVKAKEVLKSFLQSQAIM
        EDSILQSDKALTDGERAIAAERTKKEVAEKELELLRQRQKEQEQVMEAQERSFRENIAKLQEKMESEK
        EMLLREQEKMLEHKLKVQEELLIEGFREKSDMLKNEISHLREEMERTRRKPSLFGQILDTVGNAFIMI
        LPGAGKLFGVGLKFLGSLSS
    它的第一个M是在第71个氨基酸位置，它给的cds_genomic_start是从第71个M氨基酸开始的，所以不用cds_genomic_start，
    而是采用从翻译的终点溯源起点的算法.
    pep_start_pos 是query_seq（peptide） 在orf_seq（蛋白质序列）的上游位置，也就是在正向翻译的下游位置
    pep_end_pos 是query_seq（peptide） 在orf_seq（蛋白质序列）的下游位置，也就是在正向翻译的上游位置
    '''
    if strand == "+":
        exon_coordinates =  [[-exon_end, -exon_start] for [exon_start,exon_end] in reverse_sequence(exon_coordinates)]
        
        '''
        exon_coordinates 进行了取负反向，使得其翻译方向也变成5'-3'
        例如这样的PB.13x的exon坐标 [[4000, 4699], [6000, 6040], [7010, 7023], [9000, 10000]]，假如它是“-”则从右往左，
        它的转向取负后：           [[-10000, -9000], [-7023, -7010], [-6040, -6000], [-4699, -4000]]
        这样它“+”链都能使用check_trans_state函数
        '''
        cds_genomic_end = -1 * cds_genomic_end
        '''
        cds_genomic_end 是在基因组中的翻译终止坐标，在反向坐标,后面要再反向回来。
        '''
        
        cds_genomic_start_speci = -1 * find_amino_acid_position(exon_coordinates, cds_genomic_end, pep_start_pos, -1)
        cds_genomic_end_speci = -1 * find_amino_acid_position(exon_coordinates, cds_genomic_end, pep_end_pos, -1)

    elif strand == "-":

        cds_genomic_end =  cds_genomic_end

        cds_genomic_end_speci =  find_amino_acid_position(exon_coordinates, cds_genomic_end, pep_end_pos, 1)
        cds_genomic_start_speci = find_amino_acid_position(exon_coordinates, cds_genomic_end, pep_start_pos, 1)
            
    return info, cds_genomic_start_speci, cds_genomic_end_speci, query_seq

def parse_gtf_file(file_path):
    gtf_dict = {}
    with open(file_path, 'r') as gtf_file:
        for line in gtf_file:
            if line.startswith('#'):
                continue
            fields = line.strip().split('\t')
            chrom, source, feature, start, end, score, strand, frame, attributes = fields
            isoform_id = attributes.split('"')[1]
            if isoform_id in gtf_dict.keys():
                gtf_dict[isoform_id].append([int(start), int(end)])
            else:
                gtf_dict[isoform_id] = []
                
    return gtf_dict

def process_files(faa_file, isoform_info_file, isoform_gtf, output_file):

    ## 解析isoform_classcical.txt 文件
    isoform_info = {}
    with open(isoform_info_file, "r") as file:
        header = file.readline().strip().split("\t")
        for line in file:
            fields = line.strip().split("\t")
            if fields[45] != "ORF_seq" and fields[45] != "NA":
                isoform_id = fields[0]
                isoform_info[isoform_id] = dict(zip(header, fields))
    #print(isoform_info.keys())

    ##解析gtf文件
    gtf_dict = parse_gtf_file(isoform_gtf)
    
    with open(output_file, "w") as out_file:
        out_file.write("\t".join(header + ["read_id", "CDS_genomic_start_common", "CDS_genomic_end_common", "sequence_common"]) + "\n")
        for record in SeqIO.parse(faa_file, "fasta"):
            read_id = record.id.split("_")[0]
            read_seq = str(record.seq)
            if read_id in isoform_info:  # Check if read_id exists in isoform_info
                read_seq = str(record.seq)
                info = isoform_info[read_id]
                exon_coordinates = gtf_dict[read_id]
                #print(exon_coordinates)
                result_info, cds_genomic_start_speci, cds_genomic_end_speci, sequence_speci = find_isoform_info(read_seq, info, exon_coordinates)
                out_file.write("\t".join([str(result_info[col]) for col in header] + [read_id, str(cds_genomic_start_speci), str(cds_genomic_end_speci), sequence_speci]) + "\n")
            else:
                print(f"Warning: No information available for {read_id}")  # Optionally, log the missing read_id


In [7]:
fasta_file = case_sepecific_faa6
isoform_info_file = "_".join([casefaa.split(".faa")[0],"classification.txt"])
isoform_gtf = ".".join([casefaa.split(".faa")[0],"gtf"])
output_file = ".".join([casefaa.split(".faa")[0],"common_append.txt"])
process_files(fasta_file, isoform_info_file, isoform_gtf, output_file)

-40191553 -40189653 -40189229
-40191553 -40189653 -40189229
-223775836 -223774834 -223769975
-223775836 -223774834 -223769975
225845536 225846932 225855909
225845536 225846932 225855909
73007217 73008292 73043934
73007217 73008292 73043934
-119942143 -119940586 -119933010
-119942143 -119940586 -119933010
10797050 10797881 10799990
10797050 10797881 10799990
10797050 10797881 10799989
10797050 10797881 10799989
-34663279 -34658832 -34651623
-34663279 -34658832 -34651623
-49764740 -49762873 -49758953
-49764740 -49762873 -49758953
56176808 56178093 56178780
56176808 56178093 56178780
-109486603 -109486471 -109483665
-109486603 -109486471 -109483665
74848244 74848701 74853944
74848244 74848701 74853944
-53491645 -53490130 -53465575
-53491645 -53490130 -53465575
-41770902 -41770802 -41766267
-41770902 -41770802 -41766267
-15631099 -15630776 -15627163
-15631099 -15630776 -15627163
-86138188 -86137469 -86127380
-86138188 -86137469 -86127380
-108474008 -108472827 -108470709
-108474008 -1084728

## 04.查找多样本间共享表位（备用）

In [8]:

# 定义一个函数用于将序列切分成窗口
def sliding_window(sequence, window_size):
    for i in range(len(sequence) - window_size + 1):
        yield sequence[i:i+window_size], i

# 读取fasta文件,并进行滑窗处理,对于相同的多肽序列只保留一个
def process_fasta_file(file_path, window_size=8):
    sequences = defaultdict(list)
    seen_peptides = set()

    for record in SeqIO.parse(file_path, "fasta"):
        sequence_id = record.id
        sequence = str(record.seq)

        if not sequence:
            continue

        for peptide, start in sliding_window(sequence, window_size):
            if peptide not in seen_peptides:
                seen_peptides.add(peptide)
                sequences[peptide].append((sequence_id, start, len(sequence)))  # 添加序列长度信息

    return sequences

# 找到在所有样本中共同出现的多肽序列
def find_common_peptides(all_sequences):
    common_peptides = set.intersection(*[set(sequences.keys()) for sequences in all_sequences])
    return common_peptides

# 根据共同的多肽序列构建每个样本的common_sequences
def build_common_sequences(all_sequences, common_peptides):
    common_sequences = []
    for sequences in all_sequences:
        common_seq = defaultdict(list)
        for peptide in common_peptides:
            if peptide in sequences:
                common_seq[peptide] = sequences[peptide]
            else:
                print("error: not in common_seq --> "+peptide)
        common_sequences.append(common_seq)
        #print(common_sequences)
    return common_sequences

# 合并来自同一个序列的多肽序列
def merge_sequences_by_id(sample_comm_sequences):
    #print(sample_comm_sequences)
    merged_sequences = []
    merged = defaultdict(list)
    
    for peptide, occurrences in sample_comm_sequences.items():
        #print(peptide, occurrences)
        sequence_id, start, length = occurrences[0]
        merged[sequence_id].append((peptide, start, length))

    for sequence_id, peptides in merged.items():
        merged_peptides = merge_peptides_by_coordinates(peptides, max_distance=8)
        merged_sequences.append((sequence_id, merged_peptides))

    return merged_sequences

# 合并多肽序列根据坐标
def merge_peptides_by_coordinates2(peptides, max_distance):
    merged_peptides = []
    peptides.sort(key=lambda x: x[1])  # 按启始坐标排序

    current_merged = []
    for peptide, start, length in peptides:
        if not current_merged:
            current_merged.append((peptide, start, start + len(peptide)))
        else:
            last_peptide, last_start, last_end = current_merged[-1]
            if start < last_end:
                overlap = last_end - start
                current_merged[-1] = (last_peptide[:-overlap] + peptide, last_start, start + len(peptide))
            else:
                current_merged.append((peptide, start, start + len(peptide)))

    merged_peptides = [''.join(seq for seq, _, _ in current_merged)]
    return merged_peptides
    
def merge_peptides_by_coordinates(peptides, max_distance):
    merged_peptides = []
    peptides.sort(key=lambda x: x[1])  # 按启始坐标排序

    current_merged = []
    for peptide, start, length in peptides:
        if not current_merged:
            current_merged.append((peptide, start, start + len(peptide)))
        else:
            last_peptide, last_start, last_end = current_merged[-1]
            if start < last_end:
                overlap = last_end - start
                current_merged[-1] = (last_peptide[:-overlap] + peptide, last_start, start + len(peptide))
            else:
                current_merged.append((peptide, start, start + len(peptide)))

    merged_peptides = current_merged
    return merged_peptides

# 输出合并后的序列到fasta文件
def write_merged_sequences2(merged_sequences, output_file):
    with open(output_file, "w") as f:
        for i, (sequence_id, merged_peptides) in enumerate(merged_sequences, start=1):
            for j, (peptide, start, end) in enumerate(merged_peptides, start=1):
                read_id = f"{sequence_id}_{i}_{j}"
                f.write(f">{read_id}\n{peptide}\n")
def write_merged_sequences(merged_sequences, output_file):
    with open(output_file, "w") as f:
        for i, (sequence_id, merged_peptides) in enumerate(merged_sequences, start=1):
            for j, (peptide, start, end) in enumerate(merged_peptides, start=1):
                read_id = f"{sequence_id}_{i}_{j}_{start}_{end}"
                f.write(f">{read_id}\n{peptide}\n")

# 主函数
def main(fasta_files, window_size=8, output_file="merged.fasta"):
    all_sequences = [process_fasta_file(file_path, window_size) for file_path in fasta_files]
    common_peptides = find_common_peptides(all_sequences)
    #print(common_peptides)
    common_sequences = build_common_sequences(all_sequences, common_peptides)
    #print(common_sequences[1])
    merged_sequences = [merge_sequences_by_id(sample_comm_sequences) for sample_comm_sequences in common_sequences]
    #print(merged_sequences)
    for i, sequences in enumerate(merged_sequences, start=1):
        output_file_path = f"{output_file.rsplit('.', 1)[0]}_{i}.fasta"
        write_merged_sequences(sequences, output_file_path)


In [9]:
# 多个case样本中查找shared的neo protein
#case1faa = "/home/jovyan/work/09.data_STAD_sdfyy/04.Result.LSTVs/CCSW-01-T-1/results/05.Isoform_Novel/STAD-T1.isoform_classification.filtered_lite.specific4.faa"
#case2faa = "/home/jovyan/work/09.data_STAD_sdfyy/04.Result.LSTVs/CCSW-01-T-2/results/05.Isoform_Novel/STAD_T2.isoform_classification.filtered_lite.specific4.faa"
#case3faa = "/home/jovyan/work/09.data_STAD_sdfyy/04.Result.LSTVs/CCSW-01-T-3/results/05.Isoform_Novel/STAD_T3.isoform_classification.filtered_lite.specific4.faa"
#outpath2 = "/home/jovyan/work/09.data_STAD_sdfyy/04.Result.LSTVs/"
#fasta_files = [case1faa, case2faa, case3faa]
#main(fasta_files, output_file=os.path.join(outpath2, "merged.fasta"))